In [52]:
import matplotlib.pyplot as plt
import polars as pl

import numpy as np
import pandas as pd

import polars as pl
import seaborn as sns

%reload_ext autoreload
%autoreload 2

In [53]:
df_train = pl.read_parquet('../experiments/datasets/master_card_basic/train.parquet')
df_test = pl.read_parquet('../experiments/datasets/master_card_basic/test.parquet')

In [54]:
# df_train

In [55]:
from sklearn.linear_model import LogisticRegression


target = ['is_fraud']
X_feautures = ['amount', 'channel', 'currency', 'device']
categorical_columns = ['channel', 'currency', 'device']
X_feautures = ['amount', 'user_id', 'lat', 'long']
categorical_columns = ['user_id', ]

X_train, y_train = df_train[X_feautures], df_train[target]
X_train = X_train.with_columns(pl.col(c).to_physical() for c in categorical_columns)

X_test, y_test = df_test[X_feautures], df_test[target]
X_test = X_test.with_columns(pl.col(c).to_physical() for c in categorical_columns)

In [60]:
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


X_train

transformer = make_column_transformer(
    (StandardScaler(), ['amount']),
    (OneHotEncoder(sparse_output=True), categorical_columns)
)
pipe = make_pipeline(transformer, LGBMClassifier(class_weight ='balanced'))
pipe.fit(X_train, y_train.to_numpy().flatten())

[LightGBM] [Info] Number of positive: 35208, number of negative: 381534
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016303 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26931
[LightGBM] [Info] Number of data points in the train set: 416742, number of used features: 13339
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['amount']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['user_id'])])),
                ('lgbmclassifier', LGBMClassifier(class_weight='balanced'))])

In [61]:
y_hat = pipe.predict(X_test)
y_hat

/home/quak/development/mastercard/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


array([0, 1, 1, ..., 0, 0, 1], shape=(83258,))

In [62]:
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, roc_auc_score


{
    "accuracy": accuracy_score(y_test, y_hat),
    "auc": roc_auc_score(y_test, y_hat),
    "f1_score": f1_score(y_test, y_hat),
    'average_precision': average_precision_score(y_test, y_hat),
}


{'accuracy': 0.8084988829902232,
 'auc': np.float64(0.498023903344972),
 'f1_score': 0.09971767363071711,
 'average_precision': np.float64(0.08621080439461805)}

In [63]:

y_hat = pipe.predict(X_train)


{
    "accuracy": accuracy_score(y_train, y_hat),
    "auc": roc_auc_score(y_train, y_hat),
    "f1_score": f1_score(y_train, y_hat),
    'average_precision': average_precision_score(y_train, y_hat),
}


/home/quak/development/mastercard/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


{'accuracy': 0.8205148509149546,
 'auc': np.float64(0.5450163104265328),
 'f1_score': 0.16735498090902007,
 'average_precision': np.float64(0.09582669115676182)}